# Practica 11
- **Alumna:** Enya Quetzalli Gómez Rodríguez *(Eduardo Gómez Rodríguez)*
- **Profesora:** Olga Kolesnikova
- **Escuela:** Escuela Superior de Cómputo del IPN
- **Grupo:** 3CV9
- **Semestre:** 2020/2

## Relaciones sintagmáticas con Laplace-Smoothing
**Instrucciones:**
    Utilizar el práctica 10, pero implementando el cálculo de probabilidades con Laplace Smoothing

In [1]:
import nltk
import numpy
import math
from nltk.stem import SnowballStemmer

In [2]:
def cleanHtml(txt):
	from bs4 import BeautifulSoup
	return BeautifulSoup(txt,'lxml').get_text().lower()

In [3]:
def splitText(txt):
	return txt.replace('/', ' ').replace('.', ' ').replace('-', ' ')

In [4]:
def deleteTrash(txt):
	import re
	good = {'\n'}
	for i in "abcdefghijklmnopqrstuvwxyz áéíóúñü":
		good.add(i)
	ans = ""
	for c in txt:
		if c in good:
			ans += c
	return ans

In [5]:
def splitToSentences(txt):
    tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')
    return tokenizer.tokenize(txt)

In [6]:
def deleteStopwords(txt):
	from nltk.corpus import stopwords
	ans = []
	stp = stopwords.words()
	for w in txt:
		if w not in stp:
			ans.append(w)
	return ans

In [7]:
def getVocabulary(sentences):
    tokens = set()
    for sent in sentences:
        for token in sent:
            tokens.add(token)
    return sorted(tokens)

In [8]:
def getFrequencies(sentences, vocabulary):
    tokens = dict()
    
    for token in vocabulary:
        tokens[token] = 0
    
    for sent in sentences:
        for token in sent:
            tokens[token] += 1
            
    return tokens

In [9]:
def getContexts(wordList, windowSize=4):
    contexts = dict()
    index = 0
    for index in range(len(wordList)):
        word = wordList[index]
        if word not in contexts:
            contexts[word] = []
        start = max(0, index - windowSize)
        end = min(index + windowSize, len(wordList) - 1)
        for i in range(start, end + 1):
            if i != index:
                contexts[word].append(wordList[i])
    return contexts

In [10]:
def loadFromJson(name):
    import json
    with open("./files/"+name, 'r', encoding="utf8") as f:
        return json.load(f)

In [11]:
def dumpToJson(obj, name):
    import json
    with open("./files/"+name, 'w', encoding="utf8") as outfile:
        json.dump(obj, outfile, indent=4)

In [12]:
def replaceWithStems(tokens):
    ss = SnowballStemmer("spanish")
    stematized_tokens = []
    for word in tokens:
        stematized_tokens.append(ss.stem(word))
    return stematized_tokens

In [13]:
def replacWithLemmas(tokens):
    lemmas = loadFromJson("lemmatized_tokens_generate")
    lemmatized_tokens = []
    for word in tokens:
        if word in lemmas.keys():
            lemmatized_tokens.append(lemmas[word])
        else:
            lemmatized_tokens.append(word)
    return lemmatized_tokens

In [14]:
def getTagger():
    patterns = [
        (r'.*o$', 'n'),  # noun masculine singular
        (r'.*os$', 'n'), # noun masculine plural
        (r'.*a$', 'n'),  # noun femenine singular
        (r'.*as$', 'n')  # noun femenine plural
    ]
    regexTagger = nltk.RegexpTagger(patterns, nltk.DefaultTagger('s'))
    unigramTagger = nltk.UnigramTagger(nltk.corpus.cess_esp.tagged_sents(), None, regexTagger)
    return unigramTagger

In [15]:
def mixTags(tokens, tokenTags):
    taggedTokens = list()
    for i in range(0,len(tokens)):
        taggedTokens.append((tokens[i], tokenTags[i][1]))
    return taggedTokens

In [16]:
def normalizeSencences(sent):
    tagger = getTagger()
    sentences = []
    for s in sent:
        cleanSentence = deleteTrash(splitText(s))
        normalizedTokens = deleteStopwords(nltk.word_tokenize(cleanSentence))
        tokenTags = tagger.tag(normalizedTokens)
        stematizedTokens = replaceWithStems(normalizedTokens)
        tokens = mixTags(stematizedTokens, tokenTags)
        sentences.append(tokens)
    return sentences

In [17]:
f = open("./files/e961024.htm", "r", encoding="utf8")
org_txt = f.read()
f.close()

sentences = splitToSentences(cleanHtml(org_txt))
sentences = normalizeSencences(sentences)

In [18]:
vocabulary = getVocabulary(sentences)
frequencies = getFrequencies(sentences, vocabulary)

### Usando la Entropía


In [19]:
def getEntropy(tgP, wordP, target, word, n, sentences):
    prob_11 = 0
    for sent in sentences:
        if (target in sent) and (word in sent):
             prob_11 += 1
    prob_11 /= n
    
    tgQ = 1 - tgP
    wordQ = 1 - wordP
    
    prob_01 = wordP - prob_11
    prob_10 = tgP - prob_11
    prob_00 = tgQ - prob_01
    
    #print("prob => ",prob_00, prob_01, prob_10, prob_11)
    
    a,b,c,d = 0,0,0,0
    if wordQ != 0:
        if prob_00/wordQ > 0:
            a = prob_00 * math.log2(prob_00 / wordQ)
        if prob_10/wordQ > 0:
            b = prob_10 * math.log2(prob_10 / wordQ)
    if wordP != 0:
        if prob_01/wordP > 0:
            c = prob_01 * math.log2(prob_01/wordP)
        if prob_11/wordP > 0:
            d = prob_11 * math.log2(prob_11/wordP)
            
    #print("entropy => ",a,b,c,d)
            
    return (a+b+c+d)*-1

In [20]:
def getSimilarsTo(target, frequency, vocabulary, sentences):
    w = SnowballStemmer("spanish").stem(target)
    n = len(sentences)
    k = 2 #Laplace Smoothing variable
    
    targetTokens = set()
    for token in vocabulary:
        if token[0] == w:
            targetTokens.add(token)
    
    print("Checking for " + str(len(targetTokens)) + " tokens of word " + target)
    for tg in targetTokens:
        print("\t" + str(tg))
            
    similarity = dict()
    for word in vocabulary:
        similarity[word] = 0.0
    for tg in targetTokens:
        tgProb = (frequency[tg]+k)/(n+frequency[tg]*k)
        
        for word in vocabulary:
            wProb = (frequency[word]+k)/(n+frequency[tg]*k)
            entropy = getEntropy(tgProb, wProb, tg, word, n, sentences)
            similarity[word] += entropy
    
    relatedWords = []
    for token,entropy in similarity.items():
        relatedWords.append((entropy,token))
    
    return sorted(relatedWords)

In [21]:
result = getSimilarsTo("economía", frequencies, vocabulary, sentences)

Checking for 8 tokens of word economía
	('econom', 'nccp000')
	('econom', 'ncfp000')
	('econom', 'nccs000')
	('econom', 'aq0fp0')
	('econom', 'aq0fs0')
	('econom', 'aq0mp0')
	('econom', 'ncfs000')
	('econom', 'aq0ms0')


In [22]:
for r in result[:100]:
    print(str(r[0]) + ", " + str(r[1]))

0.42399246024553844, ('econom', 'ncfs000')
0.42822974710363493, ('econom', 'aq0fs0')
0.45796703422891827, ('econom', 'aq0ms0')
0.5128642448178622, ('econom', 'aq0mp0')
0.5146568179775968, ('econom', 'aq0fp0')
0.5202227051868888, ('econom', 'nccs000')
0.5302103621507589, ('econom', 'nccp000')
0.5311006299698519, ('ven', 'vmip3p0')
0.5324321609685821, ('recuper', 'ncfs000')
0.5328300127486351, ('crecimient', 'ncms000')
0.5365953606807705, ('econom', 'ncfp000')
0.5370202381498146, ('globaliz', 'ncfs000')
0.537106565969665, ('polit', 'ncfs000')
0.5372927380917684, ('pais', 'ncms000')
0.5373240347338064, ('funcionari', 'ncmp000')
0.5374473724273325, ('mundial', 'aq0cs0')
0.5374557268143445, ('financier', 'aq0mp0')
0.5376705268498186, ('perversion', 's')
0.53786814734084, ('conoc', 'ncms000')
0.5380907366992723, ('crisis', 'ncfn000')
0.5381400034191206, ('bm', 's')
0.538146611721232, ('mexican', 'aq0fs0')
0.5382471485198589, ('polit', 'aq0fp0')
0.5385303420222378, ('banc', 'ncms000')
0.53860

Al utilizar *Laplace-Smoothing*, podemos notar que algunas palabras como las variantes de economía ahorase encuentran complementamente arriba, mientras que con la entropía de probabilidad estandar algunas aparecían en diferentes posiciones a pesar de tratarse de la misma palabra. 

In [23]:
result2 = getSimilarsTo("acero", frequencies, vocabulary, sentences)

Checking for 1 tokens of word acero
	('acer', 'ncms000')


In [24]:
for r in result2[:100]:
    print(str(r[0]) + ", " + str(r[1]))

0.013764969652885355, ('acer', 'ncms000')
0.013764969652885355, ('barat', 'aq0fs0')
0.013764969652885355, ('cabl', 's')
0.013764969652885355, ('cobr', 'ncms000')
0.013764969652885355, ('codif', 's')
0.013764969652885355, ('desmaterializ', 'n')
0.013764969652885355, ('distanci', 'ncfp000')
0.013764969652885355, ('fibr', 'n')
0.013764969652885355, ('impuls', 'vmp00sm')
0.013764969652885355, ('larg', 'aq0fp0')
0.013764969652885355, ('microproces', 's')
0.013764969652885355, ('mov', 'vmn0000')
0.013764969652885355, ('noved', 's')
0.013764969652885355, ('optic', 'n')
0.013764969652885355, ('pes', 'vmp00sm')
0.013764969652885355, ('rasg', 'n')
0.013764969652885355, ('transistor', 's')
0.013764969652885355, ('traslad', 'vmp00sm')
0.013764969652885355, ('tub', 'ncmp000')
0.014031032738063744, ('aceler', 'n')
0.014031032738063744, ('cabl', 'ncms000')
0.014031032738063744, ('digital', 'aq0cs0')
0.014031032738063744, ('vaci', 'ncms000')
0.01422864397416685, ('transmision', 'ncfs000')
0.0145166035

En la prueba anterior, vimos que las primeras palabras tenían un nivel de entropía igual a 0, lo que es realmente imposible porque significa que tenemos total seguridad de que esa palabra siempre se encontrará junto a otra, sin embargo eso no falso, por lo que con *Laplace-Smoothing* podemos lograr obtener datos mas realisticos.